# HOUSEHOLD CONSUMPTIONS


# 0. Libraries export 

In [33]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#url = "https://data.open-power-system-data.org/household_data/latest/household_data_15min_singleindex.csv"
url= "household_data_15min_singleindex.csv"
df = pd.read_csv(url, sep=",")


#Las energías están en kWh 
#Los precios se tomaran en €/MWh

In [4]:
#Obtencion de los datos de precios, desde archivo, si hay tiempo mirar documentacion de la API
#url_price = "https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/export?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.12.2016+00%3A00%7CUTC%7CDAY&biddingZone.values=CTY%7C10Y1001A1001A83F!BZN%7C10Y1001A1001A82H&dateTime.timezone=UTC&dateTime.timezone_input=UTC&dataItem=ALL&timeRange=YEAR&exportType=CSV"
#https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html#_complete_parameter_list
#https://github.com/EnergieID/entsoe-py
prices_csv="Day-ahead Prices_201601010000-201701010000.csv"
df_prices = pd.read_csv(prices_csv, sep=",")


In [6]:
#Funcion para limpiar fecha, se mete en un lambda y ya no es necesario
def cutendtime(timestring):
    times = str(timestring).split("-")
    return times[0]

In [49]:
df_prices['time'] = df_prices[df_prices.columns[0]].map(lambda x: str(x).split("-")[0])
df_prices.head()


,MTU (UTC),Day-ahead Price [EUR/MWh],time
0,01.01.2016 00:00 - 01.01.2016 01:00,22.39,01.01.2016 00:00
1,01.01.2016 01:00 - 01.01.2016 02:00,20.59,01.01.2016 01:00
2,01.01.2016 02:00 - 01.01.2016 03:00,16.81,01.01.2016 02:00
3,01.01.2016 03:00 - 01.01.2016 04:00,17.41,01.01.2016 03:00
4,01.01.2016 04:00 - 01.01.2016 05:00,17.02,01.01.2016 04:00


In [8]:
df_prices.dtypes

MTU (UTC)                     object
Day-ahead Price [EUR/MWh]    float64
time                          object
dtype: object

In [9]:
df_prices['time'] = df_prices['time'].apply(pd.to_datetime)

In [20]:
df_prices_clean = pd.DataFrame({
    'time' : df_prices[df_prices.columns[2]],
    'price' : df_prices[df_prices.columns[1]]})

df_prices_clean.dtypes

price           float64
time     datetime64[ns]
dtype: object

In [66]:
df['time'] = df[df.columns[0]].apply(pd.to_datetime)
df['year'] = pd.DatetimeIndex(df['time']).year
df['month'] = pd.DatetimeIndex(df['time']).month
df.dtypes

utc_timestamp                                               object
cet_cest_timestamp                                          object
DE_KN_industrial1_grid_import                              float64
DE_KN_industrial1_pv_1                                     float64
DE_KN_industrial1_pv_2                                     float64
DE_KN_industrial2_grid_import                              float64
DE_KN_industrial2_pv_1                                     float64
DE_KN_industrial2_storage_charge                           float64
DE_KN_industrial2_storage_decharge                         float64
DE_KN_industrial3_area_offices                             float64
DE_KN_industrial3_area_room_chemistry                      float64
DE_KN_industrial3_area_room_clean                          float64
DE_KN_industrial3_area_room_printing1                      float64
DE_KN_industrial3_area_room_printing2                      float64
DE_KN_industrial3_compressor                               flo

In [39]:
df2 = pd.DataFrame({
    'time' : df['time'],
    'utc' : df['utc_timestamp']})

df2.sample(20)

,time,utc
47626,2016-08-23 13:45:00+00:00,2016-08-23T13:45:00Z
17042,2015-10-09 23:45:00+00:00,2015-10-09T23:45:00Z
12868,2015-08-27 12:15:00+00:00,2015-08-27T12:15:00Z
42678,2016-07-03 00:45:00+00:00,2016-07-03T00:45:00Z
31408,2016-03-07 15:15:00+00:00,2016-03-07T15:15:00Z
36673,2016-05-01 11:30:00+00:00,2016-05-01T11:30:00Z
56450,2016-11-23 11:45:00+00:00,2016-11-23T11:45:00Z
7011,2015-06-27 12:00:00+00:00,2015-06-27T12:00:00Z
30100,2016-02-23 00:15:00+00:00,2016-02-23T00:15:00Z
21125,2015-11-21 12:30:00+00:00,2015-11-21T12:30:00Z


In [84]:
df_2016 = df[df['year']==2016]

In [85]:
df_2016['DE_KN_residential4_washing_machine'].sample(20)


40422     73.872
48738    100.972
49571    102.944
32140     45.818
30476     38.270
28449     29.867
37464     64.745
53667    115.008
26022     19.934
52752    112.696
54104    116.505
51801    109.882
47806     97.201
33880     50.195
50558    105.666
49985    105.084
41141     77.202
54186    116.516
52531    112.672
57272    124.110
Name: DE_KN_residential4_washing_machine, dtype: float64

In [87]:
df_2016.head(60)

,utc_timestamp,cet_cest_timestamp,DE_KN_industrial1_grid_import,DE_KN_industrial1_pv_1,DE_KN_industrial1_pv_2,DE_KN_industrial2_grid_import,DE_KN_industrial2_pv_1,DE_KN_industrial2_storage_charge,DE_KN_industrial2_storage_decharge,DE_KN_industrial3_area_offices,...,DE_KN_residential6_dishwasher,DE_KN_residential6_freezer,DE_KN_residential6_grid_export,DE_KN_residential6_grid_import,DE_KN_residential6_pv,DE_KN_residential6_washing_machine,interpolated_values,time,year,month
25011,2016-01-01T00:00:00Z,2016-01-01T01:00:00+0100,24432.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.091,-241.018,NaN,NaN,NaN,6.932,NaN,2016-01-01 00:00:00,2016,1
25012,2016-01-01T00:15:00Z,2016-01-01T01:15:00+0100,24433.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.091,-241.013,NaN,NaN,NaN,6.932,NaN,2016-01-01 00:15:00,2016,1
25013,2016-01-01T00:30:00Z,2016-01-01T01:30:00+0100,24434.754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.091,-241.013,NaN,NaN,NaN,6.932,NaN,2016-01-01 00:30:00,2016,1
25014,2016-01-01T00:45:00Z,2016-01-01T01:45:00+0100,24436.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.091,-240.999,NaN,NaN,NaN,6.932,DE_KN_residential5_dishwasher,2016-01-01 00:45:00,2016,1
25015,2016-01-01T01:00:00Z,2016-01-01T02:00:00+0100,24438.258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.091,-240.999,NaN,NaN,NaN,6.932,DE_KN_residential5_dishwasher,2016-01-01 01:00:00,2016,1
25016,2016-01-01T01:15:00Z,2016-01-01T02:15:00+0100,24442.254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.091,-240.993,NaN,NaN,NaN,6.932,NaN,2016-01-01 01:15:00,2016,1
25017,2016-01-01T01:30:00Z,2016-01-01T02:30:00+0100,24446.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.091,-240.985,NaN,NaN,NaN,6.932,NaN,2016-01-01 01:30:00,2016,1
25018,2016-01-01T01:45:00Z,2016-01-01T02:45:00+0100,24448.254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.091,-240.985,NaN,NaN,NaN,6.932,NaN,2016-01-01 01:45:00,2016,1
25019,2016-01-01T02:00:00Z,2016-01-01T03:00:00+0100,24449.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.091,-240.971,NaN,NaN,NaN,6.932,NaN,2016-01-01 02:00:00,2016,1
25020,2016-01-01T02:15:00Z,2016-01-01T03:15:00+0100,24451.504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.091,-240.971,NaN,NaN,NaN,6.932,NaN,2016-01-01 02:15:00,2016,1


In [ ]:



#Unir ambos dataframes por la fecha para incluir los precios en el dataframe principal




In [75]:
from datetime import datetime
#dates=df['time'][(df['year']==2016) & (df['month']==5)]
num_list = list(range)
values = df['DE_KN_residential3_washing_machine']
plt.scatter(values)


TypeError: scatter() missing 1 required positional argument: 'y'

In [78]:
df.index

RangeIndex(start=0, stop=63939, step=1)

In [5]:
#Definición de un dataframe con los nombres de las columnas para poder trabajarlos
col_names= pd.DataFrame(list(df),columns=['col_name'])
col_names.head()

,col_name
0,utc_timestamp
1,cet_cest_timestamp
2,DE_KN_industrial1_grid_import
3,DE_KN_industrial1_pv_1
4,DE_KN_industrial1_pv_2


In [108]:
#Funciones para extraer informacion de las columnas
#La nomneclatura que siguen las columnas separa los conceptos por _ y la casa esta en el 3er campo

def gethousenumber(col_name):
    fields = str(col_name).split("_")
    house_field = 2
    return fields[house_field][-1] if len(fields)>2 else 0

def getconcept(col_name):
    fields = str(col_name).split("_")
    fields_num = len(fields)
    fixed_fields = 3
    field = list(range(fixed_fields,fields_num))
    return ''.join([fields[num] for num in field])

In [111]:
col_names['house_number'] = col_names['col_name'].apply(gethousenumber)
col_names['concept'] = col_names['col_name'].apply(getconcept)
col_names.tail(15)

,col_name,house_number,concept
58,DE_KN_residential4_pv,4,pv
59,DE_KN_residential4_refrigerator,4,refrigerator
60,DE_KN_residential4_washing_machine,4,washingmachine
61,DE_KN_residential5_dishwasher,5,dishwasher
62,DE_KN_residential5_grid_import,5,gridimport
63,DE_KN_residential5_refrigerator,5,refrigerator
64,DE_KN_residential5_washing_machine,5,washingmachine
65,DE_KN_residential6_circulation_pump,6,circulationpump
66,DE_KN_residential6_dishwasher,6,dishwasher
67,DE_KN_residential6_freezer,6,freezer


In [113]:
col_names.groupby('concept').count()
#col_names2=col_names.assign(household=)

,col_name,house_number
concept,,
,3,3
areaoffices,1,1
arearoomchemistry,1,1
arearoomclean,1,1
arearoomprinting1,1,1
arearoomprinting2,1,1
circulationpump,3,3
compressor,1,1
coolingaggregate,1,1


In [ ]:
#Crear lista con los columnas para mantener del dataframe de origen, por casas. 

In [ ]:
#Dataframeprueba
df.columns['utc_timestamp','cet_cest_timestamp',]